## Visualizing TALOS

In [1]:
import rospy
from sensor_msgs.msg import JointState
from std_msgs.msg import Header
import numpy as np
import roslib
import tf

#### Start Instruction

roslaunch talos_description upload.launch  # upload talos urdf to ros_param server

rosrun robot_state_publisher robot_state_publisher

rosrun rviz rviz -d ./rviz_config.rviz #change to rviz config locations

#### Setting up ROS variables 

In [2]:
pub = rospy.Publisher('joint_states', JointState, queue_size=10)
rospy.init_node('joint_state_publisher')
rate = rospy.Rate(200) # 10hz

config_str = JointState()
config_str.header = Header()
config_str.header.stamp = rospy.Time.now()
config_str.name = ['leg_left_1_joint','leg_left_2_joint','leg_left_3_joint','leg_left_4_joint','leg_left_5_joint',
'leg_left_6_joint','leg_right_1_joint','leg_right_2_joint','leg_right_3_joint',
'leg_right_4_joint','leg_right_5_joint','leg_right_6_joint','torso_1_joint',
'torso_2_joint','arm_left_1_joint','arm_left_2_joint','arm_left_3_joint',
'arm_left_4_joint','arm_left_5_joint','arm_left_6_joint','arm_left_7_joint',
'gripper_left_joint','arm_right_1_joint','arm_right_2_joint','arm_right_3_joint',
'arm_right_4_joint','arm_right_5_joint','arm_right_6_joint','arm_right_7_joint',
'gripper_right_joint','head_1_joint','head_2_joint']
br = tf.TransformBroadcaster()

#### Publishing the world frame w.r.t. the stairs

In [3]:
br.sendTransform((0,0,0),
                 (0,0,0,1),
                 rospy.Time.now(),
                 "stairs_link",
                 "world"
                 )
rate.sleep()

#### Loading the file

In [4]:
file_name = 'talos_circle/5'
root_file = '../samples/'

f = open(root_file + file_name + '/talos_flatGround_config.csv','rb')
traj = []
for line in f.readlines():
    traj.append(np.array([float(l) for l in line.split()])[np.newaxis,:])
print len(traj)

2948


#### Sending to rviz

In [5]:
for traj_i in traj:
    config_str.header.stamp = rospy.Time.now()
    config_str.position = traj_i[0,8:].tolist()  #the joint configurations
    config_str.velocity = []
    config_str.effort = []
    
    br.sendTransform((traj_i[0,1:4].tolist()), #root position
                 (traj_i[0,4:8].tolist()),   #root orientation
                 rospy.Time.now(),
                 "base_link",
                 "world"
                 )

    pub.publish(config_str)
    rate.sleep()

### Loading the contact information

In [6]:
import numpy as np
import pickle
from pinocchio.libpinocchio_pywrap import SE3
from crocoddyl import loadTalos, loadTalosLegs, loadHyQ, m2a

In [7]:
##################Extract Contact Sequence from File#######################
FILENAME = '../samples/talos_circle/5'
FACTOR_TIME = 1.
ROBOT = loadTalosLegs()

from locomote import ContactSequenceHumanoid
cs = ContactSequenceHumanoid(0)
cs.loadFromXML(FILENAME + '/contact_sequence_trajectory.xml', 'contact_sequence')
raw_phases = cs.contact_phases

rfs = []
lfs = []
phase_durations = []
num_phases = len(raw_phases)
for i,cur_phase in enumerate(raw_phases):
    rfs.append(cur_phase.RF_patch)
    lfs.append(cur_phase.LF_patch)
    phase_durations.append(cur_phase.time_trajectory[-1] - cur_phase.time_trajectory[0])
    print('Phase {}'.format(i))
    print('Right foot contact is {} at {}'.format(cur_phase.RF_patch.active, np.dot(cur_phase.RF_patch.placement,cur_phase.RF_patch.contactModelPlacement)))
    print('Left foot contact is {} at {}'.format(cur_phase.LF_patch.active, np.dot(cur_phase.LF_patch.placement,cur_phase.LF_patch.contactModelPlacement)))

#Convert durations to number of timesteps for each phase
timesteps = np.array(phase_durations)/0.005
timesteps = timesteps.astype(int)
print timesteps

Phase 0
Right foot contact is True at   R =
           1 -1.34086e-19  7.85045e-17
           0     0.999999     0.001708
-7.85046e-17    -0.001708     0.999999
  p =   0.964865   0.142576 0.00400528

Left foot contact is True at   R =
           1 -1.34086e-19  7.85045e-17
           0     0.999999     0.001708
-7.85046e-17    -0.001708     0.999999
  p =   0.964865   0.312576 0.00400528

Phase 1
Right foot contact is False at   R =
           1 -1.34086e-19  7.85045e-17
           0     0.999999     0.001708
-7.85046e-17    -0.001708     0.999999
  p =   0.964865   0.142576 0.00400528

Left foot contact is True at   R =
           1 -1.34086e-19  7.85045e-17
           0     0.999999     0.001708
-7.85046e-17    -0.001708     0.999999
  p =   0.964865   0.312576 0.00400528

Phase 2
Right foot contact is True at   R =
    0.998118   -0.0613284 -0.000100939
   0.0613285     0.998116    0.0017043
-3.77258e-06  -0.00170728     0.999999
  p =   0.781742  0.0908579 0.00399947

Left foot co

#### Visualize contacts in rviz using tf

In [8]:
from scipy.spatial.transform import Rotation as R
rate = rospy.Rate(1)
for i in range(num_phases):
    RF = rfs[i]
    LF = lfs[i]
    if RF.active:
        pose = np.dot(RF.placement, RF.contactModelPlacement)
        pose_trans = np.array(pose.translation).flatten()
        pose_quat = R.from_dcm(pose.rotation).as_quat()
        br.sendTransform(pose_trans.tolist(),pose_quat.tolist(), rospy.Time.now(),"RF_" + str(i), 'world')
    if LF.active:
        pose = np.dot(LF.placement, LF.contactModelPlacement)
        pose_trans = np.array(pose.translation).flatten()
        pose_quat = R.from_dcm(pose.rotation).as_quat()
        br.sendTransform(pose_trans.tolist(),pose_quat.tolist(), rospy.Time.now(),"LF_" + str(i), 'world')

    pub.publish(config_str)
    rate.sleep()